In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import threading 
import datetime
import importlib

In [2]:
from dnnTorerosModel import *

2022-05-01 23:38:11.723246: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-01 23:38:11.723341: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/wesselweernink/VENVS/pic_env/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [3]:
#importlib.reload(tm)

In [4]:
nasdaq_financials = pd.read_csv("comp_all_fin_nasdaq.csv")

In [5]:
nasdaq_financials

,Symbol,Date,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest,Net Income,Selling General Administrative,Gross Profit,Ebit,...,Discontinued Operations_lag_2,Net Income From Continuing Ops_lag_1,Net Income From Continuing Ops_lag_2,Net Income Applicable To Common Shares_lag_1,Net Income Applicable To Common Shares_lag_2,Close_lag_1,Close_lag_2,Volume_lag_1,Volume_lag_2,FutureDirection
0,AAME,2020-12-31,0.000000e+00,0,1.550300e+07,0.000000e+00,1.216900e+07,0.000000e+00,2.866100e+07,1.711300e+07,...,0.0,-3.860000e+05,-7.040000e+05,-7.850000e+05,-1.103000e+06,1.948998,2.365385,3400.0,5900.0,1.195531
1,AAOI,2020-12-31,4.339300e+07,0,-5.122400e+07,0.000000e+00,-5.845200e+07,5.599000e+07,5.054100e+07,-4.884200e+07,...,0.0,-6.604900e+07,-2.146000e+06,-6.604900e+07,-2.146000e+06,11.880000,15.430000,559800.0,774100.0,0.603995
2,AAON,2020-12-31,0.000000e+00,0,1.019750e+08,0.000000e+00,7.900900e+07,5.637700e+07,1.558490e+08,9.747300e+07,...,0.0,5.371100e+07,4.232900e+07,5.371100e+07,4.232900e+07,48.837917,34.423103,178900.0,154800.0,1.198429
3,AAPL,2020-09-26,1.875200e+10,0,6.709100e+10,0.000000e+00,5.741100e+10,1.991600e+10,1.049560e+11,6.628800e+10,...,0.0,5.525600e+10,5.953100e+10,5.525600e+10,5.953100e+10,53.674576,54.525856,101408000.0,91717600.0,1.316931
4,AATC,2020-12-31,3.336000e+06,0,6.010000e+05,0.000000e+00,1.063000e+06,6.444000e+06,1.038800e+07,6.080000e+05,...,0.0,6.995000e+06,1.862000e+06,6.995000e+06,1.862000e+06,4.252031,4.214569,2400.0,1500.0,1.472303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2359,YI,2019-12-31,6.190200e+07,0,-5.018270e+08,-2.733000e+06,-4.996060e+08,4.640630e+08,3.618700e+07,-4.935130e+08,...,0.0,-3.820410e+08,-2.493270e+08,-3.800910e+08,-2.485800e+08,6.140000,13.800000,4700.0,1302200.0,1.032689
2360,YMAB,2020-12-31,9.258300e+07,0,-1.193370e+08,0.000000e+00,-1.193370e+08,4.589900e+07,1.854700e+07,-1.199350e+08,...,0.0,-8.102800e+07,-4.327400e+07,-8.102800e+07,-4.327400e+07,31.250000,20.340000,133600.0,160900.0,0.327409
2361,YNDX,2019-12-31,2.920900e+10,0,2.285500e+10,3.450700e+10,1.282600e+10,5.015500e+10,1.196030e+11,2.546200e+10,...,0.0,4.425800e+10,9.081000e+09,4.598400e+10,9.201000e+09,27.350000,33.639999,931200.0,2087000.0,1.599908
2362,YORW,2019-12-31,0.000000e+00,0,1.664200e+07,0.000000e+00,1.440200e+07,9.968000e+06,4.119100e+07,2.233500e+07,...,0.0,1.337600e+07,1.297400e+07,1.337600e+07,1.297400e+07,30.746683,31.291149,17900.0,39100.0,1.026672


In [6]:
nasdaq_financials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2364 entries, 0 to 2363
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   Symbol                                        2364 non-null   object 
 1   Date                                          2364 non-null   object 
 2   Research Development                          2364 non-null   float64
 3   Effect Of Accounting Charges                  2364 non-null   int64  
 4   Income Before Tax                             2364 non-null   float64
 5   Minority Interest                             2364 non-null   float64
 6   Net Income                                    2364 non-null   float64
 7   Selling General Administrative                2364 non-null   float64
 8   Gross Profit                                  2364 non-null   float64
 9   Ebit                                          2364 non-null   f

In [7]:
finan_stcks = nasdaq_financials.copy() 

In [8]:
finan_stcks["PredictionGroup"] = np.where( finan_stcks["FutureDirection"]>= 1.15, 4, None )
finan_stcks["PredictionGroup"] = np.where( finan_stcks["FutureDirection"]< 1.15, 3, finan_stcks["PredictionGroup"] )
finan_stcks["PredictionGroup"] = np.where( finan_stcks["FutureDirection"]< 1.04, 2, finan_stcks["PredictionGroup"])
finan_stcks["PredictionGroup"] = np.where( finan_stcks["FutureDirection"]< 0.96, 1, finan_stcks["PredictionGroup"])
finan_stcks["PredictionGroup"] = np.where( finan_stcks["FutureDirection"]< 0.85, 0, finan_stcks["PredictionGroup"])

In [9]:
split = int(len(nasdaq_financials) * 0.8)
split

1891

In [10]:
train = finan_stcks.iloc[:split].copy()

In [11]:
train

,Symbol,Date,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest,Net Income,Selling General Administrative,Gross Profit,Ebit,...,Net Income From Continuing Ops_lag_1,Net Income From Continuing Ops_lag_2,Net Income Applicable To Common Shares_lag_1,Net Income Applicable To Common Shares_lag_2,Close_lag_1,Close_lag_2,Volume_lag_1,Volume_lag_2,FutureDirection,PredictionGroup
0,AAME,2020-12-31,0.000000e+00,0,1.550300e+07,0.0,1.216900e+07,0.000000e+00,2.866100e+07,1.711300e+07,...,-3.860000e+05,-7.040000e+05,-7.850000e+05,-1.103000e+06,1.948998,2.365385,3400.0,5900.0,1.195531,4
1,AAOI,2020-12-31,4.339300e+07,0,-5.122400e+07,0.0,-5.845200e+07,5.599000e+07,5.054100e+07,-4.884200e+07,...,-6.604900e+07,-2.146000e+06,-6.604900e+07,-2.146000e+06,11.880000,15.430000,559800.0,774100.0,0.603995,0
2,AAON,2020-12-31,0.000000e+00,0,1.019750e+08,0.0,7.900900e+07,5.637700e+07,1.558490e+08,9.747300e+07,...,5.371100e+07,4.232900e+07,5.371100e+07,4.232900e+07,48.837917,34.423103,178900.0,154800.0,1.198429,4
3,AAPL,2020-09-26,1.875200e+10,0,6.709100e+10,0.0,5.741100e+10,1.991600e+10,1.049560e+11,6.628800e+10,...,5.525600e+10,5.953100e+10,5.525600e+10,5.953100e+10,53.674576,54.525856,101408000.0,91717600.0,1.316931,4
4,AATC,2020-12-31,3.336000e+06,0,6.010000e+05,0.0,1.063000e+06,6.444000e+06,1.038800e+07,6.080000e+05,...,6.995000e+06,1.862000e+06,6.995000e+06,1.862000e+06,4.252031,4.214569,2400.0,1500.0,1.472303,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,SCHL,2020-05-31,0.000000e+00,0,-8.970000e+07,1400000.0,-4.380000e+07,7.051000e+08,6.961000e+08,-7.050000e+07,...,1.560000e+07,-5.000000e+06,1.560000e+07,-5.000000e+06,31.710461,42.495613,490900.0,134300.0,1.198008,4
1887,SCHN,2020-08-31,0.000000e+00,0,-1.939000e+06,3729000.0,-4.145000e+06,1.878030e+08,2.086180e+08,2.081500e+07,...,5.857000e+07,1.594430e+08,5.634500e+07,1.564510e+08,20.740690,23.944056,114900.0,450000.0,2.452324,4
1888,SCKT,2019-12-31,3.893563e+06,0,5.057140e+05,0.0,2.865860e+05,5.600710e+06,1.010064e+07,6.063700e+05,...,-5.711410e+05,-1.430731e+06,-5.711410e+05,-1.430731e+06,1.500000,3.600000,25000.0,20400.0,1.478261,4
1889,SCOR,2020-12-31,3.609700e+07,0,-4.701600e+07,0.0,-4.791800e+07,1.321440e+08,1.808560e+08,-1.460400e+07,...,-3.389960e+08,-1.592680e+08,-3.389960e+08,-1.592680e+08,4.940000,14.430000,1109600.0,172400.0,1.341365,4


In [12]:
test = nasdaq_financials.iloc[split:].copy()

In [13]:
columns_use = list(nasdaq_financials.columns)

In [14]:
avoid = ["Symbol", "Date", "FutureDirection", "PredictionGroup"]


In [15]:
#using reversed here to avoid problem where python skips over certain elements after deleting items because index is
# changed. Using reversed makes sure that all elements are checked. By deleting from the end of the list the index 
# of other elements in the list don't get shifted to fill the gap that would appear in the index.
for col in reversed(columns_use):
    if col in avoid:
        columns_use.remove(col)

In [16]:
train = train[columns_use]

In [17]:
len(columns_use)

72

In [18]:
#columns_use.remove("FutureDirection")

In [19]:
mu, std = train.mean(), train.std()

In [20]:
std

Research Development                            1.917989e+09
Effect Of Accounting Charges                    0.000000e+00
Income Before Tax                               3.794960e+09
Minority Interest                               3.550979e+09
Net Income                                      2.911191e+09
                                                    ...     
Net Income Applicable To Common Shares_lag_2    2.104158e+09
Close_lag_1                                     1.252084e+02
Close_lag_2                                     5.910353e+02
Volume_lag_1                                    6.454486e+06
Volume_lag_2                                    4.706799e+06
Length: 72, dtype: float64

In [21]:
mu

Research Development                            2.061252e+08
Effect Of Accounting Charges                    0.000000e+00
Income Before Tax                               4.009679e+08
Minority Interest                               1.803793e+08
Net Income                                      3.283229e+08
                                                    ...     
Net Income Applicable To Common Shares_lag_2    2.508422e+08
Close_lag_1                                     3.653849e+01
Close_lag_2                                     6.099136e+01
Volume_lag_1                                    7.319414e+05
Volume_lag_2                                    9.292109e+05
Length: 72, dtype: float64

In [22]:
train_stand = (train - mu)/ std

In [23]:
train_stand = train_stand.fillna(0)

In [24]:
pd.set_option("display.max_columns", None)

In [25]:
train_stand

,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest,Net Income,Selling General Administrative,Gross Profit,Ebit,Operating Income,Other Operating Expenses,Interest Expense,Extraordinary Items,Non Recurring,Other Items,Income Tax Expense,Total Revenue,Total Operating Expenses,Cost Of Revenue,Total Other Income Expense Net,Discontinued Operations,Net Income From Continuing Ops,Net Income Applicable To Common Shares,Close,Volume,Research Development_lag_1,Research Development_lag_2,Effect Of Accounting Charges_lag_1,Effect Of Accounting Charges_lag_2,Income Before Tax_lag_1,Income Before Tax_lag_2,Minority Interest_lag_1,Minority Interest_lag_2,Net Income_lag_1,Net Income_lag_2,Selling General Administrative_lag_1,Selling General Administrative_lag_2,Gross Profit_lag_1,Gross Profit_lag_2,Ebit_lag_1,Ebit_lag_2,Operating Income_lag_1,Operating Income_lag_2,Other Operating Expenses_lag_1,Other Operating Expenses_lag_2,Interest Expense_lag_1,Interest Expense_lag_2,Extraordinary Items_lag_1,Extraordinary Items_lag_2,Non Recurring_lag_1,Non Recurring_lag_2,Other Items_lag_1,Other Items_lag_2,Income Tax Expense_lag_1,Income Tax Expense_lag_2,Total Revenue_lag_1,Total Revenue_lag_2,Total Operating Expenses_lag_1,Total Operating Expenses_lag_2,Cost Of Revenue_lag_1,Cost Of Revenue_lag_2,Total Other Income Expense Net_lag_1,Total Other Income Expense Net_lag_2,Discontinued Operations_lag_1,Discontinued Operations_lag_2,Net Income From Continuing Ops_lag_1,Net Income From Continuing Ops_lag_2,Net Income Applicable To Common Shares_lag_1,Net Income Applicable To Common Shares_lag_2,Close_lag_1,Close_lag_2,Volume_lag_1,Volume_lag_2
0,-0.107469,0.0,-0.101573,-0.050797,-0.108599,-0.146040,-0.132305,-0.116679,-0.118591,-0.026060,0.133914,0.0,0.0,0.0,-0.086158,-0.127966,-0.122576,-0.100554,0.021131,-0.038020,-0.101349,-0.108054,-0.301133,-0.095569,-0.107417,-0.106231,0.0,0.0,-0.074255,-0.118837,-0.061572,-0.063249,-0.076861,-0.119567,-0.163325,-0.170194,-0.148346,-0.163680,-0.129006,-0.131347,-0.133707,-0.138574,-0.027797,-0.026273,0.149471,0.100508,0.0,0.0,0.0,0.0,0.0,0.0,-0.089185,-0.057147,-0.138588,-0.147594,-0.130906,-0.139082,-0.108115,-0.115928,0.057556,0.053248,-0.029633,-0.021381,-0.065559,-0.118526,-0.075576,-0.119737,-0.276255,-0.099192,-0.112874,-0.196165
1,-0.084845,0.0,-0.119156,-0.050797,-0.132858,-0.130738,-0.130042,-0.138232,-0.137600,-0.030542,0.124405,0.0,0.0,0.0,-0.082144,-0.126246,-0.117440,-0.099490,0.020503,-0.038020,-0.125058,-0.132180,-0.249176,-0.043675,-0.081716,-0.071911,0.0,0.0,-0.089126,-0.122247,-0.061572,-0.063249,-0.101221,-0.120251,-0.145507,-0.150806,-0.144115,-0.151672,-0.148032,-0.136096,-0.151144,-0.143317,-0.033873,-0.035662,0.139823,0.102378,0.0,0.0,0.0,0.0,0.0,0.0,-0.067617,-0.064317,-0.138976,-0.142301,-0.128368,-0.132354,-0.111065,-0.115202,0.057334,0.057393,-0.029633,-0.021381,-0.087755,-0.119214,-0.099782,-0.120233,-0.196940,-0.077087,-0.026670,-0.032955
2,-0.107469,0.0,-0.078787,-0.050797,-0.085640,-0.130632,-0.119152,-0.090419,-0.095429,-0.030542,0.137406,0.0,0.0,0.0,-0.065921,-0.113989,-0.110917,-0.088807,0.026104,-0.038020,-0.078910,-0.084953,0.214590,-0.081983,-0.107417,-0.106231,0.0,0.0,-0.054582,-0.096972,-0.061572,-0.063249,-0.056792,-0.099149,-0.146244,-0.151803,-0.134647,-0.149111,-0.103837,-0.109193,-0.110641,-0.116451,-0.033873,-0.035662,0.155357,0.104336,0.0,0.0,0.0,0.0,0.0,0.0,-0.069588,-0.044066,-0.124210,-0.131542,-0.118889,-0.125218,-0.095942,-0.101513,0.057849,0.054980,-0.029633,-0.021381,-0.047273,-0.097976,-0.055365,-0.099096,0.098232,-0.044952,-0.085683,-0.164530
3,9.669436,0.0,17.573314,-0.050797,19.608013,5.297080,10.718470,21.539521,18.982001,-0.030542,-6.649759,0.0,0.0,0.0,9.888970,11.883599,10.033390,10.262553,0.675813,-0.038020,19.168674,19.612650,0.575688,12.246201,9.496548,9.684616,0.0,0.0,19.102320,28.108656,-0.061572,-0.063249,20.422558,28.126158,6.143366,6.488728,12.571648,16.379427,23.971501,28.319274,21.953162,28.273214,-0.033873,-0.03

In [26]:
test = test[columns_use]

In [27]:
mu2, std2 = test.mean(), test.std()

In [28]:
test_stand = (test -mu2) / std2
test_stand = test_stand.fillna(0)

In [29]:
test_stand

,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest,Net Income,Selling General Administrative,Gross Profit,Ebit,Operating Income,Other Operating Expenses,Interest Expense,Extraordinary Items,Non Recurring,Other Items,Income Tax Expense,Total Revenue,Total Operating Expenses,Cost Of Revenue,Total Other Income Expense Net,Discontinued Operations,Net Income From Continuing Ops,Net Income Applicable To Common Shares,Close,Volume,Research Development_lag_1,Research Development_lag_2,Effect Of Accounting Charges_lag_1,Effect Of Accounting Charges_lag_2,Income Before Tax_lag_1,Income Before Tax_lag_2,Minority Interest_lag_1,Minority Interest_lag_2,Net Income_lag_1,Net Income_lag_2,Selling General Administrative_lag_1,Selling General Administrative_lag_2,Gross Profit_lag_1,Gross Profit_lag_2,Ebit_lag_1,Ebit_lag_2,Operating Income_lag_1,Operating Income_lag_2,Other Operating Expenses_lag_1,Other Operating Expenses_lag_2,Interest Expense_lag_1,Interest Expense_lag_2,Extraordinary Items_lag_1,Extraordinary Items_lag_2,Non Recurring_lag_1,Non Recurring_lag_2,Other Items_lag_1,Other Items_lag_2,Income Tax Expense_lag_1,Income Tax Expense_lag_2,Total Revenue_lag_1,Total Revenue_lag_2,Total Operating Expenses_lag_1,Total Operating Expenses_lag_2,Cost Of Revenue_lag_1,Cost Of Revenue_lag_2,Total Other Income Expense Net_lag_1,Total Other Income Expense Net_lag_2,Discontinued Operations_lag_1,Discontinued Operations_lag_2,Net Income From Continuing Ops_lag_1,Net Income From Continuing Ops_lag_2,Net Income Applicable To Common Shares_lag_1,Net Income Applicable To Common Shares_lag_2,Close_lag_1,Close_lag_2,Volume_lag_1,Volume_lag_2
1891,-0.099003,0.0,-0.191458,-0.065691,-0.328053,-0.078668,-0.104374,-0.100660,-0.126189,-0.108631,0.169150,0.0,0.0,0.0,-0.096254,0.068240,0.093685,0.227318,-0.170327,-1.008842,-0.226458,-0.327114,-0.154669,-0.212529,-0.109349,-0.114934,0.0,0.0,-0.051168,-0.099896,-0.059452,-0.094650,-0.048541,-0.109785,-0.081115,-0.078305,-0.097902,-0.104341,-0.054304,-0.070242,-0.078864,-0.095897,-0.113046,-0.105967,0.184072,0.198195,0.0,0.0,0.0,0.0,0.0,0.0,-0.054822,-0.048465,0.135554,0.153635,0.162402,0.179724,0.323780,0.321523,-0.022092,-0.002874,-0.026749,-0.016975,-0.049780,-0.112568,-0.048228,-0.103277,-0.046954,-0.064533,-0.215313,-0.294376
1892,-0.099003,0.0,-0.100552,-0.065691,-0.090854,-0.080265,-0.111333,-0.105086,-0.130588,-0.108631,0.227462,0.0,0.0,0.0,-0.088380,-0.108871,-0.103230,-0.085382,0.088161,-0.039347,-0.096537,-0.089813,-0.271124,-0.187483,-0.109349,-0.114934,0.0,0.0,-0.064655,-0.128012,-0.059452,-0.094650,-0.057412,-0.131134,-0.074322,-0.067970,-0.114753,-0.121934,-0.113024,-0.137494,-0.137336,-0.162951,-0.113046,-0.105967,0.269336,0.260116,0.0,0.0,0.0,0.0,0.0,0.0,-0.071080,-0.081424,-0.102656,-0.093499,-0.095098,-0.083031,-0.071412,-0.055595,0.002080,0.175269,0.009897,-0.000180,-0.062554,-0.136511,-0.057170,-0.124835,-0.152650,-0.133444,-0.022436,-0.030311
1893,-0.033943,0.0,-0.176763,-0.065691,-0.165950,-0.079402,-0.113228,-0.182615,-0.207638,-0.108631,0.226130,0.0,0.0,0.0,-0.144958,-0.151474,-0.140012,-0.160380,0.094413,-0.039347,-0.175867,-0.164898,-0.305130,-0.214777,-0.034583,-0.034508,0.0,0.0,-0.103688,-0.248794,-0.059452,-0.094650,-0.092620,-0.175200,-0.085697,-0.078116,-0.122072,-0.134803,-0.196686,-0.257416,-0.220646,-0.282522,-0.113046,-0.105967,0.267266,0.258895,0.0,0.0,0.0,0.0,0.0,0.0,-0.118385,-0.373571,-0.157477,-0.156992,-0.144473,-0.138356,-0.159195,-0.146176,0.003551,0.187835,0.009897,-0.000180,-0.099472,-0.179040,-0.092311,-0.168188,-0.115769,-0.136309,0.100771,-0.274280
1894,-0.072700,0.0,-0.178477,-0.065691,-0.188147,-0.167390,-0.160982,-0.180307,-0.205344,-0.108631,0.217935,0.0,0.0,0.0,-0.109640,-0.200142,-0.194609,-0.200558,0.083434,-0.039347,-0.199315,-0.187110,-0.424456,-0.188087,-0.090949,-0.096548,0.0,0.0,-0.092847,-0.196528,-0.059452,-0.094650,-0.080479,-0.197184,-0.186020,-0.192082,-0.176829,-0.193238,-0.180760,-0.210991,-0.204787,-0.236233,-0.

In [30]:
finan_stcks["PredictionGroup"][:split]

0       4
1       0
2       4
3       4
4       4
       ..
1886    4
1887    4
1888    4
1889    4
1890    1
Name: PredictionGroup, Length: 1891, dtype: object

In [31]:
finan_stcks["PredictionGroup"] = pd.to_numeric(finan_stcks["PredictionGroup"])

In [32]:
finan_stcks["PredictionGroup"]

0       4
1       0
2       4
3       4
4       4
       ..
2359    2
2360    0
2361    4
2362    2
2363    1
Name: PredictionGroup, Length: 2364, dtype: int64

In [33]:
finan_stcks[:split]

,Symbol,Date,Research Development,Effect Of Accounting Charges,Income Before Tax,Minority Interest,Net Income,Selling General Administrative,Gross Profit,Ebit,Operating Income,Other Operating Expenses,Interest Expense,Extraordinary Items,Non Recurring,Other Items,Income Tax Expense,Total Revenue,Total Operating Expenses,Cost Of Revenue,Total Other Income Expense Net,Discontinued Operations,Net Income From Continuing Ops,Net Income Applicable To Common Shares,Close,Volume,Research Development_lag_1,Research Development_lag_2,Effect Of Accounting Charges_lag_1,Effect Of Accounting Charges_lag_2,Income Before Tax_lag_1,Income Before Tax_lag_2,Minority Interest_lag_1,Minority Interest_lag_2,Net Income_lag_1,Net Income_lag_2,Selling General Administrative_lag_1,Selling General Administrative_lag_2,Gross Profit_lag_1,Gross Profit_lag_2,Ebit_lag_1,Ebit_lag_2,Operating Income_lag_1,Operating Income_lag_2,Other Operating Expenses_lag_1,Other Operating Expenses_lag_2,Interest Expense_lag_1,Interest Expense_lag_2,Extraordinary Items_lag_1,Extraordinary Items_lag_2,Non Recurring_lag_1,Non Recurring_lag_2,Other Items_lag_1,Other Items_lag_2,Income Tax Expense_lag_1,Income Tax Expense_lag_2,Total Revenue_lag_1,Total Revenue_lag_2,Total Operating Expenses_lag_1,Total Operating Expenses_lag_2,Cost Of Revenue_lag_1,Cost Of Revenue_lag_2,Total Other Income Expense Net_lag_1,Total Other Income Expense Net_lag_2,Discontinued Operations_lag_1,Discontinued Operations_lag_2,Net Income From Continuing Ops_lag_1,Net Income From Continuing Ops_lag_2,Net Income Applicable To Common Shares_lag_1,Net Income Applicable To Common Shares_lag_2,Close_lag_1,Close_lag_2,Volume_lag_1,Volume_lag_2,FutureDirection,PredictionGroup
0,AAME,2020-12-31,0.000000e+00,0,1.550300e+07,0.0,1.216900e+07,0.000000e+00,2.866100e+07,1.711300e+07,1.711300e+07,11548000.0,-1.610000e+06,0,0,0,3.334000e+06,1.953480e+08,1.782350e+08,1.666870e+08,-1610000.0,0.0,1.216900e+07,1.177000e+07,2.038039,7800.0,0.000000e+00,0.000000e+00,0.0,0.0,-4.070000e+05,-9.710000e+05,0.0,0.0,-3.860000e+05,-7.040000e+05,0.000000e+00,0.000000e+00,1.347700e+07,1.386100e+07,1.723000e+06,1.066000e+06,1.723000e+06,1.066000e+06,11754000.0,12795000.0,-2.130000e+06,-2.037000e+06,0.0,0.0,0.0,0.0,0.0,0.0,-2.100000e+04,-2.670000e+05,1.981790e+08,1.855530e+08,1.964560e+08,1.844870e+08,1.847020e+08,1.716920e+08,-2.130000e+06,-2.037000e+06,0.0,0.0,-3.860000e+05,-7.040000e+05,-7.850000e+05,-1.103000e+06,1.948998,2.365385,3400.0,5900.0,1.195531,4
1,AAOI,2020-12-31,4.339300e+07,0,-5.122400e+07,0.0,-5.845200e+07,5.599000e+07,5.054100e+07,-4.884200e+07,-4.884200e+07,0.0,-5.635000e+06,0,0,0,7.228000e+06,2.346230e+08,2.834650e+08,1.840820e+08,-2382000.0,0.0,-5.845200e+07,-5.845200e+07,8.510000,638400.0,4.339900e+07,4.990300e+07,0.0,0.0,-5.138700e+07,-9.778000e+06,0.0,0.0,-6.604900e+07,-2.146000e+06,5.154900e+07,4.863800e+07,4.620100e+07,8.777300e+07,-4.874700e+07,-1.076800e+07,-4.874700e+07,-1.076800e+07,0.0,0.0,-5.405000e+06,-1.106000e+06,0.0,0.0,0.0,0.0,0.0,0.0,1.466200e+07,-7.632000e+06,1.908720e+08,2.674650e+08,2.396190e+08,2.782330e+08,1.446710e+08,1.796920e+08,-2.640000e+06,9.900000e+05,0.0,0.0,-6.604900e+07,-2.146000e+06,-6.604900e+07,-2.146000e+06,11.880000,15.430000,559800.0,774100.0,0.603995,0
2,AAON,2020-12-31,0.000000e+00,0,1.019750e+08,0.0,7.900900e+07,5.637700e+07,1.558490e+08,9.747300e+07,9.747300e+07,0.0,-1.320000e+05,0,0,0,2.296600e+07,5.145510e+08,4.170780e+08,3.587020e+08,4502000.0,0.0,7.900900e+07,7.900900e+07,66.278435,172900.0,0.000000e+00,0.000000e+00,0.0,0.0,6.703100e+07,5.550000e+07,0.0,0.0,5.371100e+07,4.232900e+07,4.941600e+07,4.613700e+07,1.194250e+08,1.035330e+08,6.848500e+07,5.627200e+07,6.848500e+07,5.627200e+07,0.0,0.0,-1.320000e+05,-1.320000e+05,0.0,0.0,0.0,0.0,0.0,0.0,1.332000e+07,1.317100e+07,4.693330e+08,4.339470e+08,4.008480e+08,3.776750e+08,3.499080e+08,3.304140e+08,-1.454000e+06,-7.720000e+05,0.0,0.0,5.371100e+07,4.232900e+07,5.371100e+07,4.232900e+07,48.837917,34.423103,178900.0,154800.0,1

In [34]:
len(columns_use)

72

In [35]:
#############################################################
# creating model

set_seeds(101)
model = create_model(hl = 5, hu = 150, dropout = True, input_dim = len(columns_use), regularize = True)
model.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 400, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

2022-05-01 23:38:17.708769: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2022-05-01 23:38:17.708800: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: pop-os
2022-05-01 23:38:17.708806: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: pop-os
2022-05-01 23:38:17.708882: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.86.0
2022-05-01 23:38:17.708915: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.86.0
2022-05-01 23:38:17.708921: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.86.0
2022-05-01 23:38:17.710477: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operat

In [36]:
model.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 1ms/step - loss: 1.5616 - accuracy: 0.2142


[1.5615618228912354, 0.2141723930835724]

In [37]:
model.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 1ms/step - loss: 1.6194 - accuracy: 0.1966


[1.6194440126419067, 0.19661733508110046]

In [38]:
#keras_test = ["a", "b", "c", "a", "c"]

In [39]:
#keras_train = tf.keras.utils.to_categorical(keras_test, num_classes=3)

In [40]:
set_seeds(101)
model2 = create_model(hl = 2, hu = 200, dropout = True, input_dim = len(columns_use), regularize = True)
model2.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [41]:
model2.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 1ms/step - loss: 1.5331 - accuracy: 0.2845


[1.5331099033355713, 0.28450554609298706]

In [42]:
model2.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 1ms/step - loss: 1.5901 - accuracy: 0.2918


[1.5901466608047485, 0.29175475239753723]

In [43]:
set_seeds(121)
model3 = create_model(hl = 2, hu = 600, dropout = True, input_dim = len(columns_use), regularize = True)
model3.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [44]:
model3.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 2ms/step - loss: 1.4781 - accuracy: 0.3860


[1.4781378507614136, 0.386039137840271]

In [45]:
model3.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 2ms/step - loss: 1.8396 - accuracy: 0.3171


[1.8396271467208862, 0.31712472438812256]

In [46]:
set_seeds(121)
model4 = create_model(hl = 4, hu = 400, dropout = True, input_dim = len(columns_use), regularize = True)
model4.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [47]:
model4.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 2ms/step - loss: 1.5180 - accuracy: 0.2760


[1.517950415611267, 0.27604442834854126]

In [48]:
model4.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 2ms/step - loss: 1.6838 - accuracy: 0.2770


[1.6838209629058838, 0.27695560455322266]

In [49]:
set_seeds(121)
model5 = create_model(hl = 4, hu = 700, dropout = True, input_dim = len(columns_use), regularize = True)
model5.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [50]:
model5.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 3ms/step - loss: 1.4182 - accuracy: 0.4024


[1.4181690216064453, 0.40243256092071533]

In [51]:
model5.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 3ms/step - loss: 1.9063 - accuracy: 0.3087


[1.906275749206543, 0.3086680769920349]

In [52]:
set_seeds(121)
model6 = create_model(hl = 3, hu = 700, dropout = True, input_dim = len(columns_use), regularize = True)
model6.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 10, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [53]:
model6.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 3ms/step - loss: 1.6048 - accuracy: 0.2570


[1.604753017425537, 0.2570068836212158]

In [54]:
model6.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 3ms/step - loss: 1.6356 - accuracy: 0.2664


[1.635553002357483, 0.2663847804069519]

In [55]:
set_seeds(121)
model7 = create_model(hl = 4, hu = 1400, dropout = True, input_dim = len(columns_use), regularize = True)
model7.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [56]:
model7.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 7ms/step - loss: 1.1373 - accuracy: 0.5865


[1.1373075246810913, 0.5864621996879578]

In [57]:
model7.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 7ms/step - loss: 2.2319 - accuracy: 0.4228


[2.2318825721740723, 0.42283299565315247]

In [58]:
set_seeds(121)
model7a = create_model(hl = 4, hu = 1400, dropout = True, input_dim = len(columns_use), regularize = True)
model7a.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 100, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [59]:
model7a.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 8ms/step - loss: 1.3883 - accuracy: 0.4215


[1.3883280754089355, 0.42147013545036316]

In [60]:
model7a.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 8ms/step - loss: 2.0808 - accuracy: 0.2770


[2.080775737762451, 0.27695560455322266]

In [61]:
set_seeds(121)
model7b = create_model(hl = 4, hu = 1400, dropout = True, input_dim = len(columns_use), regularize = True)
model7b.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 100, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [62]:
model7b.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 8ms/step - loss: 1.3883 - accuracy: 0.4215


[1.3883280754089355, 0.42147013545036316]

In [63]:
model7b.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 7ms/step - loss: 2.0808 - accuracy: 0.2770


[2.080775737762451, 0.27695560455322266]

In [64]:
set_seeds(121)
model8 = create_model(hl = 2, hu = 1400, dropout = True, input_dim = len(columns_use), regularize = True)
model8.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [65]:
model8.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 5ms/step - loss: 1.3410 - accuracy: 0.4955


[1.3409981727600098, 0.49550503492355347]

In [66]:
model8.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 5ms/step - loss: 2.2446 - accuracy: 0.3552


[2.2445621490478516, 0.35517969727516174]

In [67]:
set_seeds(121)
model8a = create_model(hl = 2, hu = 1400, dropout = True, input_dim = len(columns_use), regularize = True)
model8a.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 300, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [68]:
model8a.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 5ms/step - loss: 1.2487 - accuracy: 0.5457


[1.2486741542816162, 0.5457429885864258]

In [69]:
model8a.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 4ms/step - loss: 2.5208 - accuracy: 0.3742


[2.5208232402801514, 0.37420719861984253]

In [70]:
set_seeds(121)
model9 = create_model(hl = 2, hu = 2100, dropout = True, input_dim = len(columns_use), regularize = True)
model9.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 200, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [71]:
model9.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 1s 9ms/step - loss: 1.2193 - accuracy: 0.5664


[1.219348669052124, 0.5663670301437378]

In [72]:
model9.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 8ms/step - loss: 2.4850 - accuracy: 0.4017


[2.484950304031372, 0.4016913175582886]

In [73]:
set_seeds(121)
model10 = create_model(hl = 2, hu = 3000, dropout = True, input_dim = len(columns_use), regularize = True)
model10.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 100, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [74]:
model10.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 1s 11ms/step - loss: 1.3270 - accuracy: 0.5172


[1.3270456790924072, 0.51718670129776]

In [75]:
model10.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 10ms/step - loss: 2.2251 - accuracy: 0.3869


[2.2250900268554688, 0.386892169713974]

In [76]:
set_seeds(121)
model11 = create_model(hl = 2, hu = 1400, dropout = True, input_dim = len(columns_use), regularize = True)
model11.fit(x = train_stand[columns_use], y = finan_stcks["PredictionGroup"][:split], epochs = 400, verbose = False,
          validation_split = 0.2, shuffle = False, class_weight = cw(finan_stcks))

In [77]:
model11.evaluate(train_stand[columns_use], finan_stcks["PredictionGroup"][:split])

60/60 [==============================] - 0s 5ms/step - loss: 1.1341 - accuracy: 0.5997


[1.13405179977417, 0.5996826887130737]

In [78]:
model11.evaluate(test_stand[columns_use], finan_stcks["PredictionGroup"][split:])

15/15 [==============================] - 0s 5ms/step - loss: 2.7979 - accuracy: 0.4017


[2.797945737838745, 0.4016913175582886]

In [79]:
predictions = model8.predict(train_stand[columns_use])
predictions

array([[0.12045593, 0.14809614, 0.21366185, 0.17614628, 0.3416398 ],
       [0.37535924, 0.29263648, 0.07148803, 0.05618178, 0.20433447],
       [0.11520883, 0.48630327, 0.1482311 , 0.14081445, 0.10944234],
       ...,
       [0.14909193, 0.14927384, 0.17031024, 0.16093086, 0.37039313],
       [0.30151483, 0.10830154, 0.12062905, 0.06211541, 0.40743917],
       [0.23088609, 0.16296667, 0.2168886 , 0.15622513, 0.23303346]],
      dtype=float32)

In [80]:
np.bincount(finan_stcks["PredictionGroup"])

array([ 599,  211,  128,  189, 1237])

In [81]:
sum(np.bincount(finan_stcks["PredictionGroup"]))

2364

In [82]:
np.bincount(finan_stcks["PredictionGroup"][:split])


array([ 478,  166,   99,  148, 1000])

In [83]:
grouper = list(np.bincount(finan_stcks["PredictionGroup"]))
print(grouper)
count = 0
for each in grouper:
    #each= float(each)
    grouper[count] = each/sum(np.bincount(finan_stcks["PredictionGroup"]))
    count +=1
print(grouper)

[599, 211, 128, 189, 1237]
[0.2533840947546531, 0.0892554991539763, 0.05414551607445008, 0.0799492385786802, 0.5232656514382402]


In [84]:
grouper = list(np.bincount(finan_stcks["PredictionGroup"][:split]))
print(grouper)
count = 0
for each in grouper:
    #each= float(each)
    grouper[count] = each/sum(np.bincount(finan_stcks["PredictionGroup"][:split]))
    count +=1
print(grouper)

[478, 166, 99, 148, 1000]
[0.2527763088313062, 0.08778424114225278, 0.0523532522474881, 0.07826546800634585, 0.5288207297726071]


In [85]:
grouper = list(np.bincount(finan_stcks["PredictionGroup"][split:]))
print(grouper)
count = 0
for each in grouper:
    #each= float(each)
    grouper[count] = each/sum(np.bincount(finan_stcks["PredictionGroup"][split:]))
    count +=1
print(grouper)

[121, 45, 29, 41, 237]
[0.2558139534883721, 0.09513742071881606, 0.0613107822410148, 0.08668076109936575, 0.5010570824524313]


In [86]:
list(np.bincount(finan_stcks["PredictionGroup"][split:]))


[121, 45, 29, 41, 237]

In [87]:
[print(i.shape, i.dtype) for i in model.inputs]

(None, 72) <dtype: 'float32'>


[None]

In [88]:
[print(o.shape, o.dtype) for o in model.outputs]

(None, 5) <dtype: 'float32'>


[None]

In [89]:
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

dense (None, 72) float32
dropout (None, 150) float32
dense_1 (None, 150) float32
dropout_1 (None, 150) float32
dense_2 (None, 150) float32
dropout_2 (None, 150) float32
dense_3 (None, 150) float32
dropout_3 (None, 150) float32
dense_4 (None, 150) float32
dropout_4 (None, 150) float32
dense_5 (None, 150) float32
dropout_5 (None, 150) float32
dense_6 (None, 150) float32


[None, None, None, None, None, None, None, None, None, None, None, None, None]